In [1]:
# The usual fare
# Importing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from astropy.io import fits
import matplotlib.cm as cm
import os 
import glob 
import astropy
import scipy.signal
from astropy.stats import sigma_clip
import time
from astropy import stats
from PIL import Image
import shift_methods as sm

In [4]:
def CustomBias(filelist,masterbias):
    masterbias=masterbias
    for file in filelist:
        header = fits.getheader(file)
        im = fits.getdata(file)
        new_name = 'b_'+file[6:-7]+'.fits'
        if im.shape == (4150,4150):
            #im_name = file[6:-7]
            overscan = np.mean(im[:,4100:4140])
            custom_bias = overscan * masterbias
            bias_subtracted = im - custom_bias
            bias_sub_overscan = bias_subtracted[:4110,:4090]
            fits.writeto(new_name,bias_sub_overscan, header,overwrite=True)
            print(new_name,'written.')

            
def norm_combine_flats(filelist):
    '''
    This normalizes the flat field images. Each flat will have a 
    pixel value of ~1. Once we do this, we can then median combine
    the flats to get our master flat for a given filter
    '''
    n = len(filelist)
    first_frame_data = fits.getdata(filelist[0])
    imsize_y, imsize_x = first_frame_data.shape
    fits_stack = np.zeros((imsize_y, imsize_x , n), dtype = np.float32) 
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        norm_im =  im/np.median(im)# finish new line here to normalize flats
        fits_stack[:,:,ii] = norm_im
    med_frame = np.median(fits_stack, axis=2)
    return med_frame



def FileMover(filelist,foldername):
            '''
    FileMover
    =======================
    More general version of filesorter.

    inputs
    =======================
    filelist  : The list of files you want to move to a new folder
    foldername: The name of the folder you want to move the files to

    returns
    =======================
    n/a

    '''
        for file in filelist:
            if os.path.exists(file):
                pass
            else:
                print(file + "does not exist or has already been moved")
                return
            if os.path.exists(foldername):
                pass
            else:
                print("Making new directory:" + foldername)
                os.mkdir(foldername)
            destination = foldername + '/'
            print("Moving " + file + " to: ./" + destination + file)
            os.rename(file, destination + file)  
            
            
def imageshifter(imlist):
        '''
    imageshifter
    =======================
    Shifts/aligns images
    
    inputs
    =======================
    imlist: list of images you want to align to each other
    
    returns
    =======================
    n/a
    
    files written
    =======================
    shifted image
'''
    imlist=imlist
    imlist.sort()
    # i=0
    # worklist= imlist[i:i+3]
    # for i in range(len(imlist)):
    im1,hdr1 = fits.getdata(imlist[0],header=True)
    #     # print(imlist[0])
    xshifts={}
    yshifts={}
    shift_image={}

    for index,filename in enumerate(imlist):
        imstack = np.zeros([im1.shape[0],im1.shape[1],len(imlist)])
        im = fits.getdata(filename)
        xshifts[index],yshifts[index]=sm.cross_image(im1,im)
        shift_image[index] = sm.shift_image(im,xshifts[index],yshifts[index])
        name = 's_'+filename
        fits.writeto(name,shift_image[index],hdr1,overwrite=True)
        print("Shift for image",name,"is",xshifts[index],yshifts[index])
        print('FITS file written:',name)
        print('*'*25)
    print('Shifting done!')
def ministack(imlist,obj,filt):
    imlist=shifted
    im1 = fits.getdata(shifted[0])
    i=0
    obj = obj
    filt = filt
    worklist=shifted[i:i+3]
    print('Starting Ministacks')
    print('*'*25)
    for i in range(len(shifted)):
        if worklist != []: 
            num = str(i)
            medianimage = mediancombine(worklist)
            fits.writeto(num+'_'+obj+'_'+filt+'_stack.fits',medianimage,header,overwrite=True)
            print('Wrote FITS File:',num+'_'+obj+'_'+filt+'_stack.fits')
            print('*'*25)

        else:
            break

        i+=i*i+3
        worklist=imlist[i:i+3]
    print('Done Ministacking')
    
def mediancombine(filelist):
    '''
    median combine
    =======================
    Takes in a list of frames and does a medain combination on them.
    Used for biases, master flats, and basic image combination.
    
    inputs
    =======================
    filelist : The list of files you want to median combine
    
    returns
    =======================
    med_frame: The median combined frame
    '''
    n = len(filelist)
    first_frame_data = fits.getdata(filelist[0])
    imsize_y, imsize_x = first_frame_data.shape
    fits_stack = np.zeros((imsize_y, imsize_x , n), dtype = np.float32) 
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        fits_stack[:,:,ii] = im
    med_frame = np.median(fits_stack, axis=2)
    return med_frame

In [3]:
# HL Tau
os.chdir('E:/20190122')
masterbias = fits.getdata('E:/20190121/NormMasterBias.fits')
masterhaflat = fits.getdata('E:/20190122/HaFlat.fits')
masterhaoffflat=fits.getdata('E:/20190122/HaOffFlat.fits')
allims=glob.glob('c*.fits')
hltauha = []
for im in allims:
    if fits.getheader(im)['DECSTRNG']=='+18:14:25' and fits.getheader(im)['FILTER2 ']=='200':
        hltauha.append(im)
CustomBias(hltauha,masterbias)
b_sci = glob.glob('b_*.fits')
for image in b_sci:
    data = fits.getdata(image)
    header = fits.getheader(image)
    flat_fielded = data / masterhaflat
    name = 'f_' + image
    fits.writeto(name,flat_fielded,header,overwrite=True)
    print(name + ' has been flatfielded')
f_sci = glob.glob('f_*.fits')
imageshifter(f_sci)

shifted = glob.glob('s_*.fits')


b_0110o.fits written.
b_0111o.fits written.
b_0112o.fits written.
f_b_0110o.fits has been flatfielded
f_b_0111o.fits has been flatfielded
f_b_0112o.fits has been flatfielded
Shift for image s_f_b_0110o.fits is 0.0 0.0
FITS file written: s_f_b_0110o.fits
*************************
Shift for image s_f_b_0111o.fits is -0.2216198055220957 -0.7610526546477558
FITS file written: s_f_b_0111o.fits
*************************
Shift for image s_f_b_0112o.fits is -0.8510473258043021 1.0158438092125834
FITS file written: s_f_b_0112o.fits
*************************
Stacking done!


In [10]:
ministack(shifted,'HLTau','Ha')

Starting Ministacks
*************************
Wrote FITS File: 0_HLTau_Ha_stack.fits
*************************
Done Ministacking


In [12]:
FileMover(b_sci,'b_sci_Ha')
FileMover(f_sci,'f_sci_Ha')
FileMover(shifted,'s_sci_Ha')


Making new directory:b_sci_Ha
Moving b_0110o.fits to: ./b_sci_Ha/b_0110o.fits
Moving b_0111o.fits to: ./b_sci_Ha/b_0111o.fits
Moving b_0112o.fits to: ./b_sci_Ha/b_0112o.fits


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'b_0112o.fits' -> 'b_sci_Ha/b_0112o.fits'